## Data introduction and import necessary Libraries

**Pixel Resolution:** 30m * 30m<br>
**Target:** Target data set<br>
**Point of Interest (POI):** POI Kernel Density, 
- POI_All, BandWidth range: 250 m - 2500 m, interval = 250 m<br>
- POI_Sel, BandWidth range: 250 m - 2500 m, interval = 250 m<br>
- POI_All: 1000m; POI_Sel: 500m<br>

**Road Network (RN):** Road Kernel Density
- BandWidth range: 250 m - 2500 m, interval = 250 m<br>
- BandWidth: 1250m<br>

**NTL:** Time,2019/03; NPP-VIIRS, DNB<br>
**XM_Boundary:** mask of all the layers<br>
**Train_Test:** 
- Train, data located in Off-island area of Xiamen, label = 0
- Test,  data located in Island area of Xiamen, label = 1

In [128]:
import math
import json
import copy
import time
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
from sl_1 import * # my custom module 

In [2]:
# Extract and generate headers
# extrat_begin('Target', 'begin')

## read clean data and split it

In [2]:
clean_data = pd.read_csv(r'data\ly_df_clean.csv')

In [3]:
clean_new = copy.deepcopy(clean_data[['ID','Target','Train_Test']])
clean_new.head()

,ID,Target,Train_Test
0,430,0.0,0.0
1,431,0.0,0.0
2,2361,0.0,0.0
3,2362,0.0,0.0
4,2363,0.0,0.0


In [122]:
len(clean_new) # Sample size: 1890571

1890571

In [48]:
Train = clean_data[clean_data['Train_Test'] == 0]
Test = clean_data[clean_data['Train_Test'] == 1]

##  convert pred data

In [86]:
# pred_df = pd.read_csv("data/pred_df.csv")
# # pred_new = copy.deepcopy(pred_df[~pred_df['clf_name'].isin(['Naive Bayes', 'QDA'])])
# pred_new['ID'] = list(range(0,len(pred_new)))
# pred_new.head()

In [97]:
pred_test = pd.read_csv("data/pred_test.csv")

In [103]:
def function1(a, b): # a: true target; b: predict
    '''
    a, One-dimensional array
    b，One-dimensional array
    '''
    arr1 = np.zeros(shape=(len(a),1))
    arr1[(b == 0) & (a == 0)] = 0 # TN
    arr1[(b == 1) & (a == 1)] = 1 # TP
    arr1[(b == 1) & (a == 0)] = 2 # FP
    arr1[(b == 0) & (a == 1)] = 3 # FN
    return arr1
# a = np.array([0,0,1,1])
# b = np.array([0,1,1,0])
# function1(a, b)

def function2(a, c): # a: true target; c: predict
    '''
    a, One-dimensional array
    c，n-dimensional array
    '''
    arr2 = np.zeros(c.shape)
    for i in range(c.shape[1]):
        b = c[:,i]
#         print('b',b)
        arr2[:,i] = function1(a, b).reshape(-1)
    return arr2

a = np.array(Test.Target)
c = np.array(pred_test)
d = function2(a, c)

In [101]:
d.shape

(171791, 35)

In [104]:
pred_test1 = pd.DataFrame(d)

## Genetate Index

### index test

In [105]:
pred_test1['ID'] = list(clean_new.iloc[:,0][clean_new['Train_Test'] == 1]) # Test data
pred_test1.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,ID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2452503
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2452504
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2452505
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2452506
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2452507


### index train

In [106]:
Train_1 = np.array([Train['Target']] * 35).T

In [107]:
Train_1.shape

(1718780, 35)

In [108]:
Train_2 = pd.DataFrame(Train_1)
Train_2['ID'] = list(clean_new.iloc[:,0][clean_new['Train_Test'] == 0]) # Train data
Train_2.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,ID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,430
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,431
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2361
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2362
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2363


### merge Train and test

In [109]:
Train_Test_arr = np.vstack([np.array(pred_test1).astype(np.int64),np.array(Train_2).astype(np.int64)])

In [110]:
Train_Test = pd.DataFrame(Train_Test_arr,columns= list(range(35)) + ['ID'])

In [111]:
Train_Test = Train_Test.sort_values(by="ID" , ascending=True)

In [112]:
Train_Test.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,ID
171791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,430
171792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,431
171793,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2361
171794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2362
171795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2363


### rebuild index

In [113]:
Raw_index = pd.DataFrame(list(range(3450552)),columns=['ID'])

In [114]:
data1 = Train_Test.copy()

In [115]:
data2 = pd.merge(Raw_index, data1, how='left', left_on='ID', right_on='ID', sort=True)

data3 = data2.copy().fillna(-9999)

In [130]:
data3.head()

,ID,0,1,2,3,4,5,6,7,8,...,25,26,27,28,29,30,31,32,33,34
0,0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,1,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,2,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,3,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,4,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [138]:
shape_ras = (1786, 1932) # Number of rows and columns
arr3 = np.array(data3).astype(np.int64)
j = arr3.shape[1]
for i in range(1,j):
    pred_y1 = arr3[:,i]
    
    pred_y2 = pred_y1.reshape(shape_ras)
    np.savetxt('data/pred_%s.txt' % (i), pred_y2, fmt='%0.0f') # Skip the first 6 lines
    add_begin('pred_%s' % (i))

## to ArcGIS Pro

In [ ]:
# -*- coding: utf-8 -*-

import arcpy

# To allow overwriting the outputs change the overwrite option to true.
arcpy.env.overwriteOutput = True

# Local variables:
for i in range(1,36):
    pred_1_txt = r"E:\Paper\XM_Urban_Built\Data\pred_35\pred_%s.txt" % (i)
    pred_1 = r"E:\Paper\XM_Urban_Built\XM_Urban_Built.gdb\pred_%s" % (i)
    Update_Input_Dataset_or_Feature_Class = pred_1

    # Process: ASCII to Raster
    arcpy.ASCIIToRaster_conversion(in_ascii_file=pred_1_txt, out_raster=pred_1, data_type="INTEGER")

    # Process: Define Projection
    arcpy.DefineProjection_management(in_dataset=pred_1, coor_system="PROJCS['UTM_Zone_50N',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['false_easting',500000.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',117.0],PARAMETER['scale_factor',0.9996],PARAMETER['latitude_of_origin',0.0],UNIT['Meter',1.0]]")